In [1]:
import pandas as pd
import json

### read data and conf

In [2]:
json_path = '../data/summarization/text_files'
data_file_name = 'txt_dataset_summarizer.json'
conf_file_name = 'txt_dataset_summarizer_config.json'

In [3]:
data = pd.read_json(f'{json_path}/{data_file_name}', orient='index')
with open(f'{json_path}/{conf_file_name}') as f:
    conf_json = json.load(f)

data.head(3)

,text,word_count,golden,notion,grammarly,current_doc_summarizier,current_update_summarizer,version_1_summarizer,version_2_summarizer,version_3_summarizer,version_2_turbo_summarizer,version_3_turbo_summarizer
0,(CNN)A mammoth fire broke out Friday morning i...,143,Fire breaks out at the General Electric Applia...,A massive fire broke out at the General Electr...,A massive fire broke out at the General Electr...,A large fire broke out Friday morning in the G...,A large fire broke out Friday morning in the G...,Title: Fire Breaks Out in Kentucky Industrial ...,Title: Fire Breaks Out in Kentucky Industrial ...,A fire broke out Friday morning in a Kentucky ...,Title: Mammoth fire breaks out at Kentucky ind...,Title: Mammoth fire breaks out at Kentucky ind...
1,"Hull, Leicester and Swansea City are following...",147,Hull's chief scout Stan Ternent has watched M...,"Hull, Leicester, and Swansea City are interest...","Hull, Leicester, and Swansea City are interest...","Maciej Rybus, a 25-year-old Polish internation...","Maciej Rybus, a 25-year-old Polish internation...","Title: Maciej Rybus Followed by Hull, Leiceste...","Title: Maciej Rybus Followed by Hull, Leiceste...","Hull, Leicester and Swansea City are intereste...","Title: Hull, Leicester and Swansea interested ...","Title: Hull, Leicester and Swansea interested ..."
2,Sportsmail have teamed up with Golfbidder to o...,123,Sportsmail have teamed up with Golfbidder for ...,Sportsmail and Golfbidder have partnered to of...,Sportsmail and Golfbidder are running a compet...,Sportsmail have partnered with Golfbidder to o...,Sportsmail and Golfbidder have teamed up to of...,Title: Win a Callaway Golf Prize Bundle Worth ...,Title: Win a Callaway Golf Prize Bundle Worth ...,One lucky reader has the chance to win a bundl...,Title: Win a bundle of Callaway golf clubs and...,Title: Win a bundle of Callaway golf clubs and...


In [4]:
print(data.iloc[5]['grammarly'])

Workforms has recently undergone updates, including support for the People Column and a 1:1 question format. The Connect Board Column and a submission analytics page are also coming soon. Check out the details on their website.


# Building new prompts

In [6]:
import openai
import os
from dotenv import load_dotenv
from modules.openai.OpenaiModels import OpenaiCompletionWordLimit, OpenaiChatCompletionWordLimit

load_dotenv()

azure_openai_key = os.environ.get('AZURE_OPENAI_KEY')
azure_openai_endpoint = os.environ.get('AZURE_OPENAI_ENDPOINT')

openai.api_key = azure_openai_key
openai.api_base = azure_openai_endpoint
openai.api_type = "azure"

In [64]:
system_prompt = """You are a helpful assistant. Using your deep linguistic understanding and knowledge, condense the essence of the content provided by the user in the ###input:''### section.
  1. Retain the core ideas, crucial details, and significant context while omitting any extraneous information.
  2. Please deliver a concise and coherent summary of the provided input.
  3. Respond only in string using less than {{MAX_WORD_COUNT}} words.
  4. The output structure should be as follow: Title:write here a proper title\n\nwrite here the summary
  5. DONT use 'Summary:' before writing the actual summary.
  """

user_prompt = """###input:{input_text}###
                Please summarize the input text in the required structure."""

version_3_5 = OpenaiChatCompletionWordLimit(model_name="gpt-3-5-turbo",
                                            api_version="2023-05-15",
                                            temperature=0.0,
                                            max_tokens=200,
                                            system_prompt_template=system_prompt,
                                            user_prompt_template=user_prompt,
                                            max_word_ratio=0.75)


In [66]:
for i in range(data.shape[0]):
    print(version_3_5.call_llm(data.iloc[i].text))
    print('\n----------------\n')

Title: Mammoth fire breaks out at Kentucky industrial park

A fire broke out at General Electric Appliance Park in Louisville, Kentucky, causing thick smoke and bright orange flames. Firefighters worked to contain the damage, and there were no reports of injuries or trapped individuals. The cause of the fire is unknown, and the facility is known for revitalizing manufacturing in the US.

----------------
Title: Hull, Leicester and Swansea interested in Poland's Maciej Rybus

Premier League clubs Hull, Leicester, and Swansea are monitoring the performance of Maciej Rybus, a 25-year-old winger from Poland who currently plays for Russian side Terek Grozny. Hull's chief scout, Stan Ternent, has watched him play, and Rybus has a get-out clause for £3.2 million. He has scored three goals in 27 appearances for club and country this season and impressed during Poland's 1-1 draw with Republic of Ireland last month.

----------------
Title: Win a bundle of Callaway golf clubs and accessories wor

In [18]:
# version_2_template = """Using your deep linguistic understanding and knowledge, condense the essence of the provided content.\n Retain the core ideas, crucial details, and significant context while omitting any extraneous information.\n Please deliver a concise and coherent summary of the following text.\n The summary MUST be shorter or equal to {max_words} words.\n The output MUST be a JSON object with the following schema:
# {{"title":<the title>,"body":<the summary>}}\n----------------------\n\nThe text to summarize: {input_text}\n The output:\n"""
# 
# version_2_sum = OpenaiCompletionWordLimit(
#     model_name="text-davinci-003",
#     api_version="2023-05-15",
#     temperature=0.0,
#     max_tokens=512,
#     prompt_template=version_2_template,
#     max_word_ratio=0.75
# )

Title: פעילות צבאית בבית הספר בלב עזה

סגן אלוף זיו בואניש מוביל פעילות צבאית בבית הספר בלב עזה, בו נמצאה מחרטה לייצור רקטות. צוות צבאי זיהה מחבל במבנה סמוך ובוצע ירי מסיבי על המטרה. הלוחמים מעידים על שינוי מג


In [ ]:
# try using function calls

In [40]:
system_prompt = """You are a helpful assistant. Using your deep linguistic understanding and knowledge, condense the essence of the content provided by the user in the ###input:''### section.
1. Retain the core ideas, crucial details, and significant context while omitting any extraneous information.
2. Please deliver a concise and coherent summary of the provided input.
3. Respond only in string using less than '100' words.
5. DONT use 'summary:' before writing the actual summary."""

user_prompt = """###input:{input_text}###
                Please summarize the input text in the required structure."""

# user_prompt = user_prompt.format(**{"input_text":data.iloc[0].text})

messages_vals = [
    {"role": "system", "content": 'you are an ai assistant'},
    {"role": "user", "content": 'tell me something funny'}
]

model_name = "gpt-3-5-turbo",
api_version = "2023-05-15"

response = openai.ChatCompletion.create(
    engine=model_name,
    api_version=api_version,
    messages=messages_vals,
    temperature=0.0,
    max_tokens=512
)

'{\n  "title": "Massive fire at Kentucky industrial park",\n  "summary": "A massive fire broke out at the General Electric Appliance Park in Louisville, Kentucky. The fire started early in the morning and sent thick smoke and bright orange flames into the air. Firefighters worked to contain the blaze, and there were no reports of injuries or trapped individuals. The cause of the fire is unknown, and the facility is described as revitalizing manufacturing in the United States."\n}'

In [29]:
openai.ChatCompletion.create(engine=model_name, api_version=api_version,
                             messages=[{'role': 'user', 'content': 'tell me something funny.'}])

<OpenAIObject chat.completion id=chatcmpl-8KRL1BRRECIfhSlBbRnCvhg9GasZK at 0x1751754a0> JSON: {
  "id": "chatcmpl-8KRL1BRRECIfhSlBbRnCvhg9GasZK",
  "object": "chat.completion",
  "created": 1699882339,
  "model": "gpt-35-turbo-16k",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "function_call": {
          "name": "return_summary",
          "arguments": "{\n  \"title\": \"Fire breaks out at Kentucky industrial park\",\n  \"summary\": \"A massiv

In [ ]:
"""
function getRecipe() {
    // Create prompt text with user input
    const prompt = `return a recipe for ${userInput}`;
    //Define the JSON Schema by creating a schema object
    const schema = {
      "type": "object",
      "properties": {
        "dish": {
          "type": "string",
          "description": "Descriptive title of the dish"
        },
        "ingredients": {
          "type": "array",
          "items": {"type": "string"}
        },
        "instructions": {
          "type": "array",
          "description": "Steps to prepare the recipe.",
          "items": {"type": "string"}
        }
      }
    }
    //Note the updated model and added functions and function_call lines
    //Note that we pass our schema object to parameters
    openai.createChatCompletion({
      model: "gpt-3.5-turbo-0613",
      messages: [
        { role: "system", "content": "You are a helpful recipe assistant." },
        { role: "user", content: prompt }],
      functions: [{ name: "set_recipe", parameters: schema }],
      function_call: {name: "set_recipe"}

    })
      .then((completion) => {
        // Note the updated location for the response
        const generatedText =
          completion.data.choices[0].message.function_call.arguments;
        setRecipe(JSON.parse(generatedText));
      })
      .catch((error) => {
        console.log(error);
      });
  }
"""

### Apply on dataset

In [37]:
model_name = "version_3_turbo_summarizer"

NameError: name 'version_3_5' is not defined

In [24]:
# completion api
data[model_name] = data['text'].apply(version_3_5.call_llm)

In [25]:
print(data.iloc[5][model_name])

In [37]:
# data = data.drop('larium_devinci_summarizer',axis=1)

{
    "title": "Mourinho confident ahead of Chelsea's clash with Manchester United",
    "summary": "Jose Mourinho believes Manchester United's injury crisis will not weaken their team as they head to Chelsea for tomorrow's title clash. Mourinho listed the lavish squad of Manchester United and said that they have amazing players, experience, and solutions. Chelsea can move 10 points clear of second-placed Arsenal with three points against United. Mourinho is relishing the meeting with Manchester United, saying his players require no motivation. Diego Costa will return to full training next week after making a quick recovery from a hamstring problem."
}


### update config

In [156]:
conf_json.update({model_name: version_3_5.get_config()})

In [157]:
conf_json

{'current_doc_summarizer': {'model_name': 'text-davinci-003',
  'api_version': '2023-05-15',
  'temperature': 0.0,
  'max_tokens': 200,
  'prompt_template': 'summarize the following text: {input_text}'},
 'current_update_summarizer': {'model_name': 'text-davinci-003',
  'api_version': '2023-05-15',
  'temperature': 0.0,
  'max_tokens': 200,
  'prompt_template': 'summarize the following text: {input_text}'},
 'version_1_summarizer': {'model_name': 'text-davinci-003',
  'api_version': '2023-05-15',
  'temperature': 0.0,
  'max_tokens': 200,
  'prompt_template': 'Summarize the key points of the text provided between the []. The summary MUST be concise and shorter than {max_words} words. Also, the output should be in the following structure: \n\nTitle: <the title>\n\n<the summary>.  \n----------------------\nThe text to summarize: [{input_text}]\n\nThe concise summary:\n',
  'max_word_ratio': 0.6},
 'version_2_summarizer': {'model_name': 'text-davinci-003',
  'api_version': '2023-05-15',
 

## Save data file and conf json

In [158]:
# save
data.to_json(f"{json_path}/{data_file_name}", orient='index', indent=4, force_ascii=True)
with open(f'{json_path}/{conf_file_name}', 'w', encoding='utf-8') as f:
    json.dump(conf_json, f, ensure_ascii=False, indent=4)